# ID2222 Data Mining, Homework 2
# **Discovery of Frequent Itemsets and Association Rules**

Brando Chiminelli, Tommaso Praturlon

November 21th, 2022

The goal of this notebook is to ...

## Import libraries and read the dataset

In order to run this notebook you need to import the dataset at this address (https://canvas.kth.se/courses/36211/files/5772174/download?wrap=1) in a 'data' directory.

In [108]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

PATH_TO_DATA = "../data/T10I4D100K.dat"
df_market = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")
# Delete duplicates from the dataset in the columns title and text

# Reduce dataset size for computation overload (temporary)
df_market = df_market.iloc[0:100]
print(df_market.head())
print("Number of baskets: ", len(df_market))

Data read successfully!
                                                   0
0  25 52 164 240 274 328 368 448 538 561 630 687 ...
1            39 120 124 205 401 581 704 814 825 834 
2                    35 249 674 712 733 759 854 950 
3            39 422 449 704 825 857 895 937 954 964 
4  15 229 262 283 294 352 381 708 738 766 853 883...
Number of baskets:  100


## Finding frequent itemsets with support at least s

Remind that an association rule is an implication X → Y, where X and Y are itemsets such that X∩Y=∅. Support of rule X → Y is the number of transactions that contain X⋃Y. Confidence of rule X → Y is the fraction of transactions containing X⋃Y in all transactions that contain X.

TASK

You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

The A-Priori algorithm is based on the rule of monotonic increase of the monotonicity of support: if a set I of items is frequent, then so is every subset of I.

The threshold s of the support should be set sufficiently hihg that not so many frequent itemsets are together. As a rule of thumb, s is 1% of the number of baskets.

### Optimizations
- Simple random sampling of the dataset -> only a subset of all baskets to reduce weight on memory
- PYC Algorithm + Multistage Algorithm (if necessary)

In [109]:
# DATA CLEANING
# Make the dataframe a list of list integers
baskets_ls = []
# take all the baskets with their items
df_baskets = df_market[df_market.columns[0]]

for basket in df_baskets:
    basket = basket.split() # split the string of items
    basket_ls = [] # create the single basket as list
    for item in basket:
        item = int(item) # convert an item to int
        basket_ls.append(item) # add it to the basket
    baskets_ls.append(basket_ls) # add the basket to the list

print(baskets_ls)

[[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834], [39, 120, 124, 205, 401, 581, 704, 814, 825, 834], [35, 249, 674, 712, 733, 759, 854, 950], [39, 422, 449, 704, 825, 857, 895, 937, 954, 964], [15, 229, 262, 283, 294, 352, 381, 708, 738, 766, 853, 883, 966, 978], [26, 104, 143, 320, 569, 620, 798], [7, 185, 214, 350, 529, 658, 682, 782, 809, 849, 883, 947, 970, 979], [227, 390], [71, 192, 208, 272, 279, 280, 300, 333, 496, 529, 530, 597, 618, 674, 675, 720, 855, 914, 932], [183, 193, 217, 256, 276, 277, 374, 474, 483, 496, 512, 529, 626, 653, 706, 878, 939], [161, 175, 177, 424, 490, 571, 597, 623, 766, 795, 853, 910, 960], [125, 130, 327, 698, 699, 839], [392, 461, 569, 801, 862], [27, 78, 104, 177, 733, 775, 781, 845, 900, 921, 938], [101, 147, 229, 350, 411, 461, 572, 579, 657, 675, 778, 803, 842, 903], [71, 208, 217, 266, 279, 290, 458, 478, 523, 614, 766, 853, 888, 944, 969], [43, 70, 176, 204, 227, 334, 369, 480, 513, 703, 708, 835, 874, 895], [25, 

The first pass of the A-Priori algorithm is to determine which are the frequent items as singletons. Thus creating a _frequent-items table_ hopefully smaller than the one with all the items.
C_k is the Candidate set for k items.
First iteration: find frequent items

In [110]:
from itertools import combinations
import statistics

# items must have at least a frequence of support threshold 1% of total baskets
S_THRESHOLD = 0.01*len(baskets_ls)

# dictionary containing all frequencies for frequent items
C_1 = dict()
# take all the baskets with their items
# for every basket take the item and if it already exists
# in the dictionary count +1
for basket in baskets_ls:
    for item in basket:
        C_1[item] = C_1.get(item,0) + 1 # get gives the i value, if not found, gives 0
        
# find frequency statistics among items 
min_freq = min(C_1.values())
max_freq = max(C_1.values())
median = statistics.median(C_1.values())
print("Minimum frequency: ", min_freq)
print("Maximum frequency: ", max_freq)
print("Median: ", median)

# delete non-frequent items
for item in list(C_1): # c1 is a list of dictionaries (1:6, where 1 is the value and 6 the counter)
    if C_1[item]<S_THRESHOLD:
        del C_1[item]

items = list(C_1.keys()) # list of all different frequent items
support = [C_1] # list of dictionaries
#print("Support for C_1: \n", support)
#print("List of frequent items:\n", items)

Minimum frequency:  1
Maximum frequency:  10
Median:  2.0


The second step of the algorithm is to count all the pairs that consist of two frequent items. At the end of this step, we examine the structure of counts to determine which pairs are frequent. The same steps are applied to find larger sets of frequent items.
For the Monotonicity Rule, we know that if no frequent itemsets of a certain size are found, there cannot be a larger itemset of them, therefore we can break the iteration. 
Second iteration: to find frequent combinations of items among frequent items.

In [111]:
# for every possible length of boundles, (a, b), (a, c, d), (e, f, g, w), ...
# ideally there is a number of Candidate Items Sets as big as the cardinality
# of all frequent singletons
MIN_SUPPORT = median
#for i in range(2,len(items)):
for i in range(2, 3):
    s = dict() # new support, now for doubletons, tripletons, etc. 
    # for every combinations of i items
    # count frequency of every combination among all baskets
    for combo in combinations(items,i):
        # iterate again in every basket of the original dataframe
        # must recreate the dataframe as set of int
        for basket in baskets_ls:
            # if the combination of i items is found in the basket, count+1
            if set(combo).issubset(basket):
                s[combo] = s.get(combo,0) + 1
        # once all baskets are checked
        # if there is a set for that combination and it is below threshold
        # delete it -> kkeeep  only actually frequuent items
        if s.get(combo) and s[combo]<MIN_SUPPORT:
            del s[combo]
    # if s is empty -> that combination is not present in any basket
    if not s:
        break # exit the for cycle (monotonicity rule)
    support.append(s) # add the support of multiple-tons

# Print list of all dictionaries for each combination with their frequencies
print(support)

[{25: 2, 52: 4, 164: 2, 240: 1, 274: 9, 328: 1, 368: 6, 448: 1, 538: 6, 561: 4, 630: 1, 687: 1, 730: 2, 775: 6, 825: 6, 834: 3, 39: 5, 120: 2, 124: 1, 205: 2, 401: 4, 581: 4, 704: 6, 814: 3, 35: 2, 249: 1, 674: 4, 712: 3, 733: 3, 759: 2, 854: 2, 950: 3, 422: 1, 449: 3, 857: 2, 895: 5, 937: 5, 954: 1, 964: 4, 15: 1, 229: 3, 262: 1, 283: 3, 294: 1, 352: 2, 381: 5, 708: 3, 738: 3, 766: 5, 853: 6, 883: 5, 966: 6, 978: 1, 26: 2, 104: 2, 143: 3, 320: 1, 569: 6, 620: 1, 798: 1, 7: 1, 185: 1, 214: 2, 350: 3, 529: 10, 658: 1, 682: 2, 782: 3, 809: 4, 849: 3, 947: 5, 970: 2, 979: 1, 227: 4, 390: 5, 71: 6, 192: 3, 208: 4, 272: 2, 279: 5, 280: 1, 300: 1, 333: 2, 496: 3, 530: 1, 597: 2, 618: 1, 675: 5, 720: 4, 855: 5, 914: 3, 932: 3, 183: 7, 193: 2, 217: 4, 256: 1, 276: 4, 277: 2, 374: 1, 474: 1, 483: 1, 512: 1, 626: 1, 653: 2, 706: 3, 878: 3, 939: 1, 161: 4, 175: 3, 177: 6, 424: 2, 490: 1, 571: 6, 623: 4, 795: 7, 910: 1, 960: 3, 125: 1, 130: 2, 327: 2, 698: 1, 699: 2, 839: 1, 392: 4, 461: 2, 801: 1

The other crucial part of the algorithm is to find significant rules that connect different items to each other.

In [112]:
MIN_CONFIDENCE = 50.0 #confidence is set to be at least 50% (confidence is the conditional probability of the itemset)

rules = dict()
for combo in support[-1]: #start from the last element of support, so the biggest cardinality of combo
    for item in combo:
        c = list(combo) 
        c.remove(item) #we need to remove one item from the combo to test our rule
        len_c = len(c)
        c = c[0] if len_c == 1 else tuple(c) #if we have to deal with tuple if our combo now has more than one element
        
        #now we compute the confidence that is:
        #the support of the union of the combo and the item divided by the support of the combo
        rule_1 = support[-1][combo]/support[0][item]*100 
        rule_2 = support[-1][combo]/support[len_c-1][c]*100 #we do the same with the opposite rule (rules are not symmetric)
        
        if rule_1>=MIN_CONFIDENCE: rules[f"{item}->{c}"] = rule_1
        if rule_2>=MIN_CONFIDENCE: rules[f"{c}->{item}"] = rule_2

print(rules)

{'25->52': 100.0, '52->25': 50.0, '25->730': 100.0, '730->25': 100.0, '52->368': 50.0, '52->730': 50.0, '730->52': 100.0, '52->676': 50.0, '676->52': 66.66666666666666, '411->274': 50.0, '764->274': 66.66666666666666, '213->274': 66.66666666666666, '21->274': 50.0, '32->274': 50.0, '54->274': 100.0, '136->274': 50.0, '239->274': 66.66666666666666, '753->274': 100.0, '639->274': 66.66666666666666, '521->274': 66.66666666666666, '368->947': 50.0, '947->368': 60.0, '227->538': 50.0, '172->538': 66.66666666666666, '464->538': 100.0, '561->758': 50.0, '758->561': 66.66666666666666, '561->919': 50.0, '78->775': 100.0, '781->775': 100.0, '684->775': 66.66666666666666, '834->825': 66.66666666666666, '825->39': 50.0, '39->825': 60.0, '825->704': 66.66666666666666, '704->825': 66.66666666666666, '39->704': 60.0, '704->39': 50.0, '205->401': 100.0, '401->205': 50.0, '401->12': 50.0, '12->401': 66.66666666666666, '581->814': 50.0, '814->581': 66.66666666666666, '581->895': 50.0, '581->966': 75.0, 